In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:\\Users\\bailey\\Projects\\seen\\data\\EDFacts\\FS002 DG74\\ID 74 SCH - Educational Environment by Gender by Disability.csv")

In [3]:
df = df[['LEAID', 'LEA_NAME', 'LEA_STATE', 'DISABILITY_CATEGORY', 'TOTAL_STUDENTS_REPORTED']]

In [4]:
values = df['DISABILITY_CATEGORY'].unique()
values = values[values != 'MISSING']

# Adds empty columns for each disability category
for col_name in values:
        df[col_name] = 0


def modify_row(row):
    disability_type = row['DISABILITY_CATEGORY']
    value = row['TOTAL_STUDENTS_REPORTED']
    row[disability_type] = value
    return row

df = df.apply(modify_row, axis=1)

In [5]:
"""
Split up columns into what values need to be summed or averaged across schools in the LEA.
Columns that intrisncially the same across the LEA, just grabs the first value.
"""
lea_groups = df.groupby('LEAID')

columns = np.append(values, 'TOTAL_STUDENTS_REPORTED')

sum_df = lea_groups[columns].agg('sum').reset_index()

mean_df = lea_groups[columns].agg('mean').reset_index()
# Rounds all averages.
for col in columns:
    mean_df[col] = mean_df[col].apply(lambda x: int(x))


first_df = lea_groups[['LEA_NAME']].first()

In [6]:
"""
Renaming dataframes
"""

sum_name_mapping = {old_name: old_name+'_TOTAL' for old_name in columns}
sum_name_mapping['TOTAL_STUDENTS_REPORTED'] = 'TOTAL_STUDENTS_REPORTED'
sum_df = sum_df.rename(columns=sum_name_mapping)

mean_name_mapping = {old_name: old_name+'_MEAN' for old_name in columns}
mean_name_mapping['TOTAL_STUDENTS_REPORTED'] = 'MEAN_STUDENTS_REPORTED'
mean_df = mean_df.rename(columns=mean_name_mapping)

In [7]:
m_1 = pd.merge(first_df, sum_df, on='LEAID')
merged_df = pd.merge(m_1, mean_df, on='LEAID')

In [9]:
# create a new column for each district showing the disability with highest concentration

def highest_disability(row):
    max_class = ""
    max_num = 0

    for dis_class in values:
        col_name = dis_class + "_TOTAL"
        
        if row[col_name] > max_num:
            max_class = dis_class
            max_num = row[col_name]

    row['MOST_COMMON_DISABILITY'] = max_class

    return row

merged_df = merged_df.apply(highest_disability, axis=1)

In [11]:
merged_df.to_csv('data/processed_data/IDEA_by_district.csv', index=False)